In [ ]:
!pip install transformers ipadic fugashi
!python -V

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.1 MB/s 
     |████████████████████████████████| 13.4 MB 25.8 MB/s 
     |████████████████████████████████| 615 kB 69.8 MB/s 
     |████████████████████████████████| 182 kB 81.0 MB/s 
     |████████████████████████████████| 7.6 MB 53.4 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=ed3c43e8a14090fb6e744eb45eca4b1ab52f3e080870eaf3e887f29e5f4bf585
  Stored in directory: /root/.cache/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
Successfully built ipadic
Python 3.8.16


In [ ]:
import glob
import json
import os
import pickle
import random
import re

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertModel

In [ ]:
# 関数群
# 前処理
def preprocessing(text):
    # ストップワードを定義
    with open("Japanese.txt", "r", encoding="utf-8") as f:
        stop_words = set([w.strip() for w in f] + ["する", "なる", "いる", "ある"])
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[w/:%#$&?()~.=+-…]+[:]? ", "", text)
    text = re.sub(r"(^RT )", "", text)
    text = text.lower()  # 小文字化
    text = re.sub("\r", "", text)  # \r\nをdelete
    text = re.sub("\n", "", text)  # \r\nをdelete
    text = re.sub(r"\d+", "", text)  # 数字列をdelete
    ZEN = "".join(chr(0xFF01 + i) for i in range(94))  # 全角記号一覧
    HAN = "".join(chr(0x21 + i) for i in range(94))  # 半角記号一覧
    ETC = "".join(chr(0x3000 + i) for i in range(30))  # その他主要そうな記号
    text = text.translate(str.maketrans(ZEN, HAN))  # 全角記号を半角記号に置換
    FIXED_HAN = re.sub(r"[\w]+", "", HAN)
    return re.sub("[" + "~" + "*" + "＊" + ETC + FIXED_HAN + "]", " ", text)  # 記号を消す


# JSONからdf作成
def JSONtoDF(path):
    with open(path) as f:
        j = json.load(f)
        return pd.json_normalize(data=j["data"]).drop(
            ["created_at", "edit_history_tweet_ids"], axis=1
        )


def prepareCorpus(path_list):
    res = pd.DataFrame()
    for path in path_list:
        df = JSONtoDF(path)
        df["preprocessedText"] = [preprocessing(text) for text in df["text"]]
        res = pd.concat([res, df])
    return res.reset_index(drop=True)


def get_context(token_ids, target_position, sequence_length=128):
    """
    Given a text containing a target word, return the sentence snippet which surrounds the target word
    (and the target word's position in the snippet).

    :param token_ids: list of token ids (for an entire line of text)
    :param target_position: index of the target word's position in `tokens`
    :param sequence_length: desired length for output sequence (e.g. 128, 256, 512)
    :return: (context_ids, new_target_position)
                context_ids: list of token ids for the output sequence
                new_target_position: index of the target word's position in `context_ids`
    """
    # -2 as [CLS] and [SEP] tokens will be added later; /2 as it's a one-sided window
    window_size = int((sequence_length - 2) / 2)
    context_start = max([0, target_position - window_size])
    padding_offset = max([0, window_size - target_position])
    padding_offset += max([0, target_position + window_size - len(token_ids)])
    context_ids = token_ids[context_start : target_position + window_size]
    context_ids += padding_offset * [0]
    new_target_position = target_position - context_start
    return context_ids, new_target_position


def get_usage(
    text_list=[],
    target_word=None,
    output_path="word-vectors/{}.dict".format(random.randrange(1000, 10000)),
    sequence_length=256,
    buffer_size=512,
    layer_range=(1, 14),
):
    if not target_word:
        return
    TW_token = tokenizer.encode(target_word)
    TW_token = TW_token[1 : len(TW_token) - 1]
    batches = []
    surrounding_words = []
    for text in tqdm(text_list):
        if target_word not in text:
            continue
        [
            surrounding_words.append(w)
            for w in tokenizer.tokenize(text)
            if w not in stop_words and w not in surrounding_words
        ]
        tokens = tokenizer.encode(text)
        TW_position = (
            tokens.index(TW_token[0])
            if len(TW_token) > 0 and TW_token[0] in tokens
            else None
        )
        if TW_position == None:
            continue
        # このループで取り扱っている一文に対象の単語が含まれている場合以下の処理を実行する
        input_ids, position = get_context(tokens, TW_position, sequence_length)
        batches.append(
            {
                "word": target_word,
                "position": (position, position + len(TW_token)),
                "input_ids": input_ids,
            }
        )
        if len(batches) >= buffer_size:
            break

    print("Start model fit")
    usages = []
    # すでにファイルが存在すれば続きから追記
    if os.path.exists(output_path):
        with open(output_path, "rb") as f:
            usages = pickle.load(f)
    else:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with torch.no_grad():
        input_ids_tensor = torch.tensor([b["input_ids"] for b in batches])
        outputs = model(input_ids_tensor, output_hidden_states=True)
        hidden_states = [l.detach().numpy() for l in outputs[2]]
        # outputs[2]は三次元のtensor型が13個並ぶ配列 (13, 512, 256, 768) (13, B, |s|, 768)
        # 13(隠れ12層+最終層)×文章数×単語数×768次元になる

        SW_tokens = [
            tokenizer.encode(w) + max([0, 10 - len(tokenizer.encode(w))]) * [0]
            for w in surrounding_words
        ]
        outputs = model(torch.tensor(SW_tokens), output_hidden_states=True)
        [
            usages.append(
                {
                    "word": w,
                    "vector": np.sum(
                        outputs.last_hidden_state.detach().numpy()[i, 0:10, :]
                    ),
                }
            )
            for i, w in enumerate(surrounding_words)
        ]

    print("Finish model fit: word: {} / size: {}".format(target_word, len(batches)))

    # defaultで12層すべての和をとる
    usage_vectors = np.sum(
        np.stack(hidden_states)[layer_range[0] : layer_range[1], :, :, :], axis=0
    )

    batch = {"word": "", "vector": [], "position": [], "input_ids": []}
    for i, b in enumerate(batches):
        b["vector"] = np.sum(
            usage_vectors[i, b["position"][0] : b["position"][1], :], axis=0
        )  # 文章の分散表現
        batch = {
            "word": b["word"],
            "vector": np.row_stack((batch["vector"], b["vector"]))
            if len(batch["vector"])
            else b["vector"],
            "position": batch["position"] + [b["position"]],
            "input_ids": batch["input_ids"] + [b["input_ids"]],
        }

    usages.append(batch)
    with open(output_path, "wb") as f:
        pickle.dump(usages, f)


# 未実施処理
# - 重複Tweet削除

In [ ]:
# 初期設定群
target = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(target,tokenize_chinese_chars=False)
model = BertModel.from_pretrained(target)

target_words = [
    "失笑",
    "なし崩し",
    "なしくずし",
    "御の字",
    # "姑息",
    "すべからく",
    "割愛",
    "破天荒",
    "役不足",
    "確信犯",
    "炎上",
    "草",
]
with open("Japanese.txt", "r", encoding="utf-8") as f:
  stop_words = set([w.strip() for w in f] + ["する", "なる", "いる", "ある"])

data_dir = "twitter-corpus"
path_list = glob.glob(data_dir + "/*.json")
path_list.sort()
# ターゲットとする単語や期間の整理は予めここで行う
oparation = [
    {
        "word": w,
        "year": y,
        "path_list": list(filter(lambda x: w in x and str(y) in x, path_list)),
    }
    for w in target_words
    for y in range(2007, 2021)
]
for o in oparation:
    if not len(o["path_list"]):
        continue
    output_path = "word-vectors/" + o["word"] + "-" + str(o["year"]) + ".dict"
    corpus = prepareCorpus(o["path_list"])
    get_usage(
        text_list=corpus["preprocessedText"].values.tolist(),
        target_word=o["word"],
        output_path=output_path,
        sequence_length=256,
        buffer_size=512,
        layer_range=(1, 14),
    )

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 36%|███▌      | 511/1420 [00:00<00:00, 1532.30it/s]


Start model fit
Finish model fit: word: 失笑 / size: 512


 20%|█▉        | 511/2587 [00:00<00:01, 1071.88it/s]


Start model fit
Finish model fit: word: 失笑 / size: 512


 10%|█         | 516/4972 [00:00<00:04, 1017.00it/s]


...

Start model fit
Finish model fit: word: 草 / size: 512


  5%|▌         | 522/9733 [00:00<00:04, 2017.23it/s]


Start model fit
Finish model fit: word: 草 / size: 512


  5%|▌         | 515/9600 [00:00<00:06, 1402.73it/s]


Start model fit
Finish model fit: word: 草 / size: 512


In [ ]:
!zip -r /content/word-vectors.zip /content/word-vectors
from google.colab import files
files.download("/content/word-vectors.zip")

  adding: content/word-vectors/ (stored 0%)
  adding: content/word-vectors/すべからく-2010.dict (deflated 29%)
  adding: content/word-vectors/なし崩し-2018.dict (deflated 48%)
  adding: content/word-vectors/確信犯-2018.dict (deflated 76%)
  adding: content/word-vectors/役不足-2007.dict (deflated 21%)
  adding: content/word-vectors/御の字-2015.dict (deflated 25%)
  adding: content/word-vectors/すべからく-2018.dict (deflated 41%)
  adding: content/word-vectors/御の字-2011.dict (deflated 24%)
  adding: content/word-vectors/なし崩し-2008.dict (deflated 21%)
  adding: content/word-vectors/炎上-2014.dict (deflated 31%)
  adding: content/word-vectors/失笑-2009.dict (deflated 23%)
  adding: content/word-vectors/役不足-2013.dict (deflated 27%)
  adding: content/word-vectors/失笑-2017.dict (deflated 25%)
  adding: content/word-vectors/確信犯-2008.dict (deflated 23%)
  adding: content/word-vectors/なしくずし-2019.dict (deflated 47%)
  adding: content/word-vectors/失笑-2011.dict (deflated 25%)
  adding: content/word-vectors/割愛-2017.dict (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>